# 02 — Routing Condicional

**Objetivo**: Clasificar queries del usuario y dirigirlas al especialista correcto (Batman, Spider-Man, o Comparacion).

## Contenido
1. Router LLM con Pydantic
2. Especialistas por personaje
3. StateGraph con conditional edges
4. Visualizacion del grafo

In [ ]:
import os
import json
from typing import Literal, TypedDict, Annotated
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

print("=" * 60)
print("ROUTING CONDICIONAL")
print("=" * 60)

## 1. Router LLM con Pydantic

El router clasifica la query del usuario en una de 3 rutas.

In [ ]:
# ============================================================
# MODELO DE ROUTING
# ============================================================

class RouteDecision(BaseModel):
    """Decision de routing para una query sobre comics."""
    ruta: Literal["batman", "spiderman", "comparacion"] = Field(
        description="Ruta seleccionada segun el contenido de la query"
    )
    confianza: float = Field(
        description="Nivel de confianza en la decision (0.0 a 1.0)",
        ge=0.0, le=1.0
    )
    razonamiento: str = Field(
        description="Breve explicacion de por que se eligio esta ruta"
    )


router_llm = llm.with_structured_output(RouteDecision)

ROUTER_PROMPT = """Eres un clasificador de queries sobre comics. 
Clasifica la query del usuario en una de estas rutas:
- "batman": preguntas especificas sobre Batman, Gotham, sus villanos, etc.
- "spiderman": preguntas especificas sobre Spider-Man, Nueva York, sus villanos, etc.
- "comparacion": preguntas que involucran a ambos personajes o comparaciones.
"""

# Test del router
test_queries = [
    "Como se convirtio Bruce Wayne en Batman?",
    "Que poderes tiene Spider-Man?",
    "Quien es mas inteligente, Batman o Spider-Man?",
]

print("Test del Router:")
for query in test_queries:
    decision = router_llm.invoke([
        SystemMessage(content=ROUTER_PROMPT),
        HumanMessage(content=query),
    ])
    print(f"\n  Query: {query}")
    print(f"  Ruta: {decision.ruta} (confianza: {decision.confianza:.1%})")
    print(f"  Razon: {decision.razonamiento}")

## 2. Especialistas por Personaje

Cada especialista tiene acceso a los datos de su personaje y un system prompt especifico.

In [ ]:
# ============================================================
# ESPECIALISTAS
# ============================================================

def cargar_comics(personaje: str) -> list[dict]:
    """Carga los comics de un personaje."""
    ruta = f"../data/{personaje}_comics.json"
    with open(ruta) as f:
        return json.load(f)


def crear_contexto(comics: list[dict], max_chars: int = 3000) -> str:
    """Crea contexto resumido de los comics."""
    contexto = []
    chars = 0
    for comic in comics:
        texto = f"[{comic['titulo']}]: {comic['contenido'][:300]}"
        if chars + len(texto) > max_chars:
            break
        contexto.append(texto)
        chars += len(texto)
    return "\n\n".join(contexto)


PROMPTS_ESPECIALISTAS = {
    "batman": "Eres un experto en Batman y el universo DC. Responde basandote en los datos proporcionados. Se preciso y cita arcos narrativos especificos.",
    "spiderman": "Eres un experto en Spider-Man y el universo Marvel. Responde basandote en los datos proporcionados. Se preciso y cita arcos narrativos especificos.",
    "comparacion": "Eres un experto en comics que compara Batman y Spider-Man. Usa evidencia de ambos universos para dar respuestas balanceadas.",
}

print("Especialistas configurados:")
for nombre in PROMPTS_ESPECIALISTAS:
    print(f"  - {nombre}")

## 3. StateGraph con Conditional Edges

```
         ┌─────────┐
         │  START   │
         └────┬────┘
              │
         ┌────▼────┐
         │  Router   │
         └────┬────┘
              │
     ┌────────┼────────┐
     ▼        ▼        ▼
┌─────────┐ ┌──────┐ ┌────────────┐
│ Batman  │ │Spider│ │Comparacion │
└────┬────┘ └──┬───┘ └─────┬──────┘
     │         │            │
     └─────────┼────────────┘
               ▼
         ┌───────────┐
         │    END     │
         └───────────┘
```

In [ ]:
# ============================================================
# STATEGRAPH CON ROUTING
# ============================================================

class RouterState(TypedDict):
    messages: Annotated[list, add_messages]
    ruta: str
    confianza: float


def nodo_router(state: RouterState) -> dict:
    """Clasifica la query y decide la ruta."""
    query = state["messages"][-1].content
    decision = router_llm.invoke([
        SystemMessage(content=ROUTER_PROMPT),
        HumanMessage(content=query),
    ])
    return {"ruta": decision.ruta, "confianza": decision.confianza}


def nodo_batman(state: RouterState) -> dict:
    """Especialista en Batman."""
    comics = cargar_comics("batman")
    contexto = crear_contexto(comics)
    query = state["messages"][-1].content if isinstance(state["messages"][-1], HumanMessage) else state["messages"][-1].content
    response = llm.invoke([
        SystemMessage(content=f"{PROMPTS_ESPECIALISTAS['batman']}\n\nContexto:\n{contexto}"),
        HumanMessage(content=query),
    ])
    return {"messages": [response]}


def nodo_spiderman(state: RouterState) -> dict:
    """Especialista en Spider-Man."""
    comics = cargar_comics("spiderman")
    contexto = crear_contexto(comics)
    query = state["messages"][-1].content if isinstance(state["messages"][-1], HumanMessage) else state["messages"][-1].content
    response = llm.invoke([
        SystemMessage(content=f"{PROMPTS_ESPECIALISTAS['spiderman']}\n\nContexto:\n{contexto}"),
        HumanMessage(content=query),
    ])
    return {"messages": [response]}


def nodo_comparacion(state: RouterState) -> dict:
    """Especialista en comparaciones."""
    batman_comics = cargar_comics("batman")
    spider_comics = cargar_comics("spiderman")
    contexto = "BATMAN:\n" + crear_contexto(batman_comics, 1500) + "\n\nSPIDER-MAN:\n" + crear_contexto(spider_comics, 1500)
    query = state["messages"][-1].content if isinstance(state["messages"][-1], HumanMessage) else state["messages"][-1].content
    response = llm.invoke([
        SystemMessage(content=f"{PROMPTS_ESPECIALISTAS['comparacion']}\n\nContexto:\n{contexto}"),
        HumanMessage(content=query),
    ])
    return {"messages": [response]}


def decidir_ruta(state: RouterState) -> str:
    """Funcion de routing condicional."""
    return state["ruta"]


# Construir grafo
graph = StateGraph(RouterState)
graph.add_node("router", nodo_router)
graph.add_node("batman", nodo_batman)
graph.add_node("spiderman", nodo_spiderman)
graph.add_node("comparacion", nodo_comparacion)

graph.add_edge(START, "router")
graph.add_conditional_edges("router", decidir_ruta, {
    "batman": "batman",
    "spiderman": "spiderman",
    "comparacion": "comparacion",
})
graph.add_edge("batman", END)
graph.add_edge("spiderman", END)
graph.add_edge("comparacion", END)

app = graph.compile()
print("Grafo de routing compilado exitosamente")

In [ ]:
# ============================================================
# PRUEBAS DEL ROUTING
# ============================================================

queries_test = [
    "Que paso en The Killing Joke?",
    "Como funciona el sentido aracnido?",
    "Quien tiene mejor filosofia de vida, Batman o Spider-Man?",
]

print("=" * 60)
print("PRUEBAS DE ROUTING")
print("=" * 60)

for query in queries_test:
    result = app.invoke({"messages": [HumanMessage(content=query)]})
    ruta = result.get("ruta", "?")
    respuesta = result["messages"][-1].content
    
    print(f"\nQuery: {query}")
    print(f"Ruta: {ruta}")
    print(f"Respuesta: {respuesta[:300]}...")
    print("-" * 40)

## 4. Visualizacion del Grafo

LangGraph soporta exportar el grafo como Mermaid diagram.

In [ ]:
# ============================================================
# VISUALIZACION MERMAID
# ============================================================

mermaid = app.get_graph().draw_mermaid()
print("Diagrama Mermaid del grafo:\n")
print(mermaid)

## Takeaways

1. **Routing** clasifica el input antes de procesarlo, permitiendo especialistas dedicados
2. **Pydantic** garantiza que la decision de routing sea estructurada y validada
3. **Conditional edges** en LangGraph mapean strings a nodos destino
4. El routing agrega una llamada LLM adicional, pero mejora la calidad de respuesta
5. La confianza del router puede usarse para implementar fallbacks